# Consigna: Práctica 7 Ejercicio 4

### Implementación y evaluación con GRU

Modifique la implementación previa para **usar GRU (Gated Recurrent Units)**.
Luego, ejecute una serie de **experimentos** para evaluar las diferencias en desempeño respecto de la implementación anterior.

In [1]:
import numpy as np

# Entrada/Salida de datos
data = open('input.txt', 'r').read() # archivo de texto plano de entrada
chars = list(set(data)) # caracteres unicos en el archivo de entrada
data_size, vocab_size = len(data), len(chars) # calcula la cantidad total de caracteres y el tamaño del vocabulario
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) } # diccionario: caracter -> indice
ix_to_char = { i:ch for i,ch in enumerate(chars) } # diccionario: indice -> caracter

data has 57028 characters, 103 unique.


In [2]:
# Hiperparametros comunes a RNN y GRU
hidden_size = 125 # tamaño de la capa oculta (cantidad de neuronas)
seq_length = 100 # numero de pasos para desenrollar la RNN (longitud de secuencia)
learning_rate = 0.001
np.random.seed(42) # semilla fija para reproducibilidad

In [3]:
# Parametros del modelo RNN
rnn_Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # pesos: entrada -> capa oculta
rnn_Whh = np.random.randn(hidden_size, hidden_size)*0.01 # pesos: capa oculta -> capa oculta (memoria)
rnn_Why = np.random.randn(vocab_size, hidden_size)*0.01 # pesos: capa oculta -> salida
rnn_bh = np.zeros((hidden_size, 1)) # sesgo de la capa oculta
rnn_by = np.zeros((vocab_size, 1)) # sesgo de la capa de salida

# Memoria Adagrad para RNN
# Se usan para acumular los cuadrados de los gradientes y modificar la tasa de aprendizaje
rnn_mWxh = np.zeros_like(rnn_Wxh)
rnn_mWhh = np.zeros_like(rnn_Whh)
rnn_mWhy = np.zeros_like(rnn_Why)
rnn_mbh = np.zeros_like(rnn_bh)
rnn_mby = np.zeros_like(rnn_by)

In [4]:
# Funciones para el modelo RNN
def rnn_lossFun(inputs, targets, hprev):
    """
    Calcula la pérdida y los gradientes de una RNN simple.

    Parámetros:
        inputs (list[int]): índices de los caracteres de entrada.
        targets (list[int]): índices esperados como salida.
        hprev (ndarray): estado oculto previo (hidden_size, 1).

    Retorna:
        loss (float): pérdida total.
        dWxh, dWhh, dWhy, dbh, dby (ndarray): gradientes de los parámetros.
        hprev (ndarray): último estado oculto.
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # Forward pass
    # Recorre la secuencia de entrada, actualizando el estado oculto y calculando las probabilidades de salida.
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1)) # vector one-hot para el caracter de entrada
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh( # estado oculto actual
            np.dot(rnn_Wxh, xs[t]) + # influencia de la entrada actual
            np.dot(rnn_Whh, hs[t-1]) + # influencia del estado anterior
            rnn_bh # sesgo/bias
        )
        ys[t] = np.dot(rnn_Why, hs[t]) + rnn_by # salida sin normalizar
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilidades (softmax)
        loss += -np.log(ps[t][targets[t], 0]) # perdida de entropia cruzada acumulada
    
    # Backward pass
    # Calcula los gradientes de todos los parametros mediante retropropagacion en el tiempo (BPTT)
    dWxh, dWhh, dWhy = np.zeros_like(rnn_Wxh), np.zeros_like(rnn_Whh), np.zeros_like(rnn_Why)
    dbh, dby = np.zeros_like(rnn_bh), np.zeros_like(rnn_by)
    dhnext = np.zeros_like(hs[0])
    
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # gradiente de la perdida
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(rnn_Why.T, dy) + dhnext # gradiente del estado oculto
        dhraw = (1 - hs[t] * hs[t]) * dh # derivada de tanh
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(rnn_Whh.T, dhraw) # propaga el gradiente hacia atras en el tiempo

    # Limita los gradientes para evitar explosion de gradientes
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def rnn_sample(h, seed_ix, n):
    """
    Genera una secuencia de caracteres con la RNN.

    Parámetros:
        h (ndarray): estado oculto inicial.
        seed_ix (int): índice del caracter inicial.
        n (int): cantidad de caracteres a generar.

    Retorna:
        list[int]: índices de los caracteres generados.
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []

    # Genera n caracteres de salida uno por uno
    for t in range(n):
        h = np.tanh(np.dot(rnn_Wxh, x) + np.dot(rnn_Whh, h) + rnn_bh) # actualiza el estado oculto
        y = np.dot(rnn_Why, h) + rnn_by # calcula la salida
        p = np.exp(y) / np.sum(np.exp(y)) # convierte a probabilidad (softmax)
        ix = np.random.choice(range(vocab_size), p=p.ravel()) # muestra el siguiente caracter segun p
        x = np.zeros((vocab_size, 1))
        x[ix] = 1 # actualiza la entrada con el nuevo caracter
        ixes.append(ix)
    
    return ixes

In [5]:
# Parametros del modelo GRU

# Compuerta de actualizacion (update gate)
# Controla cuanto del estado anterior se conserva
gru_Wxz = np.random.randn(hidden_size, vocab_size)*0.01 # pesos: entrada -> z
gru_Whz = np.random.randn(hidden_size, hidden_size)*0.01 # pesos: estado oculto -> z
gru_bz = np.zeros((hidden_size, 1)) # sesgo de la compuerta z

# Compuerta de reset (reset gate)
# Decide cuanto del estado anterior se ignora al calcular el nuevo candidato
gru_Wxr = np.random.randn(hidden_size, vocab_size)*0.01 # pesos: entrada -> r
gru_Whr = np.random.randn(hidden_size, hidden_size)*0.01 # pesos: estado oculto -> r
gru_br = np.zeros((hidden_size, 1)) # seesgo de la compuerta r

# Candidato de estado oculto
# Representa la posible nueva informacion a incorporar en el estado
gru_Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # pesos: entrada -> ĥ
gru_Whh = np.random.randn(hidden_size, hidden_size)*0.01 # pesos: estado oculto -> ĥ
gru_bh = np.zeros((hidden_size, 1)) # sesgo del candidato ĥ

# Capa de salida
gru_Why = np.random.randn(vocab_size, hidden_size)*0.01 # pesos: estado oculto -> salida
gru_by = np.zeros((vocab_size, 1)) # sesgo de la salida

# Memoria Adagrad
# Acumulan los cuadrados de los gradientes para ajustar la tasa de aprendizaje
gru_mWxz = np.zeros_like(gru_Wxz)
gru_mWhz = np.zeros_like(gru_Whz)
gru_mbz = np.zeros_like(gru_bz)
gru_mWxr = np.zeros_like(gru_Wxr)
gru_mWhr = np.zeros_like(gru_Whr)
gru_mbr = np.zeros_like(gru_br)
gru_mWxh = np.zeros_like(gru_Wxh)
gru_mWhh = np.zeros_like(gru_Whh)
gru_mbh = np.zeros_like(gru_bh)
gru_mWhy = np.zeros_like(gru_Why)
gru_mby = np.zeros_like(gru_by)

In [6]:
# Funciones para el modelo GRU
def gru_sigmoid(x):
    """Función sigmoide para las compuertas"""
    return 1 / (1 + np.exp(-x))

def gru_lossFun(inputs, targets, hprev):
    """
    Calcula pérdida y gradientes para una GRU (forward + backward).

    Parámetros:
        inputs (list[int]): índices de entrada.
        targets (list[int]): índices objetivo.
        hprev (ndarray): estado oculto previo (hidden_size, 1).

    Devuelve:
        loss (float): pérdida total
        dWxz, dWhz, dbz,
        dWxr, dWhr, dbr,
        dWxh, dWhh, dbh,
        dWhy, dby (ndarray): gradientes en ese orden,
        hprev (ndarray): último estado oculto.
    """
    xs, hs, hs_tilde, zs, rs, ys, ps = {}, {}, {}, {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # Forward pass
    # Recorre la secuencia, calcula compuertas, candidato, nuevo estado y probabilidades
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1)) # vector one-hot de entrada
        xs[t][inputs[t]] = 1
        
        # Compuerta de actualización z
        zs[t] = gru_sigmoid(np.dot(gru_Wxz, xs[t]) + np.dot(gru_Whz, hs[t-1]) + gru_bz)
        
        # Compuerta de reset r
        rs[t] = gru_sigmoid(np.dot(gru_Wxr, xs[t]) + np.dot(gru_Whr, hs[t-1]) + gru_br)
        
        # Candidato de estado oculto ĥ
        hs_tilde[t] = np.tanh(
            np.dot(gru_Wxh, xs[t]) +
            np.dot(gru_Whh, rs[t] * hs[t-1]) +
            gru_bh
        )
        
        # Nuevo estado oculto (interpolacion entre estado anterior y candidato)
        hs[t] = (1 - zs[t]) * hs[t-1] + zs[t] * hs_tilde[t]
        
        # Calcular salida
        ys[t] = np.dot(gru_Why, hs[t]) + gru_by # puntiaciones sin normalizar
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilidades (softmax)
        
        # Calcular pérdida
        loss += -np.log(ps[t][targets[t], 0])
    
    # Backward pass (BPTT)
    dWxz = np.zeros_like(gru_Wxz)
    dWhz = np.zeros_like(gru_Whz)
    dbz = np.zeros_like(gru_bz)
    dWxr = np.zeros_like(gru_Wxr)
    dWhr = np.zeros_like(gru_Whr)
    dbr = np.zeros_like(gru_br)
    dWxh = np.zeros_like(gru_Wxh)
    dWhh = np.zeros_like(gru_Whh)
    dbh = np.zeros_like(gru_bh)
    dWhy = np.zeros_like(gru_Why)
    dby = np.zeros_like(gru_by)
    dhnext = np.zeros_like(hs[0])
    
    for t in reversed(range(len(inputs))):
        # Gradiente de la salida
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        
        dh = np.dot(gru_Why.T, dy) + dhnext # gradiente respecto al estado oculto actual
        
        # Gradiente del candidato ĥ
        dh_tilde = dh * zs[t] # la parte que paso por la compuerta z
        dh_tilde_raw = (1 - hs_tilde[t] * hs_tilde[t]) * dh_tilde # derivada de la tanh
        dbh += dh_tilde_raw
        dWxh += np.dot(dh_tilde_raw, xs[t].T)
        dWhh += np.dot(dh_tilde_raw, (rs[t] * hs[t-1]).T)
        
        # Gradiente de la compuerta de reset r
        dr = np.dot(gru_Whh.T, dh_tilde_raw) * hs[t-1]
        dr_raw = dr * rs[t] * (1 - rs[t]) # derivada sigmoide
        dbr += dr_raw
        dWxr += np.dot(dr_raw, xs[t].T)
        dWhr += np.dot(dr_raw, hs[t-1].T)
        
        # Gradiente de la compuerta de actualización z
        dz = dh * (hs_tilde[t] - hs[t-1])
        dz_raw = dz * zs[t] * (1 - zs[t]) # derivada sigmoide
        dbz += dz_raw
        dWxz += np.dot(dz_raw, xs[t].T)
        dWhz += np.dot(dz_raw, hs[t-1].T)
        
        # Gradiente hacia el estado oculto anterior
        dhnext = dh * (1 - zs[t]) # parte que viene por la interpolacion
        dhnext += np.dot(gru_Whz.T, dz_raw) # contribucion via z
        dhnext += np.dot(gru_Whr.T, dr_raw) # contribucion via r
        dhnext += np.dot(gru_Whh.T, dh_tilde_raw) * rs[t] # contribucion via el candidato
    
    # Recortar gradientes para prevenir explosion
    for dparam in [dWxz, dWhz, dbz, dWxr, dWhr, dbr, dWxh, dWhh, dbh, dWhy, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    
    return loss, dWxz, dWhz, dbz, dWxr, dWhr, dbr, dWxh, dWhh, dbh, dWhy, dby, hs[len(inputs)-1]

def gru_sample(h, seed_ix, n):
    """
    Genera una secuencia de índices usando la GRU.

    Parámetros:
        h (ndarray): estado oculto inicial.
        seed_ix (int): índice inicial.
        n (int): longitud de secuencia a generar.

    Retorna:
        list[int]: índices generados.
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []

    # Genera n indices uno por uno
    for t in range(n):
        z = gru_sigmoid(np.dot(gru_Wxz, x) + np.dot(gru_Whz, h) + gru_bz)
        r = gru_sigmoid(np.dot(gru_Wxr, x) + np.dot(gru_Whr, h) + gru_br)
        h_tilde = np.tanh(np.dot(gru_Wxh, x) + np.dot(gru_Whh, r * h) + gru_bh)
        h = (1 - z) * h + z * h_tilde
        
        y = np.dot(gru_Why, h) + gru_by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    
    return ixes

In [7]:
# Entrenamiento RNN
rnn_n = 0 # contador de iteraciones
rnn_p = 0 # puntero a la posicion de los datos
rnn_smooth_loss = -np.log(1.0/vocab_size)*seq_length # perdida inicial estimada
rnn_num_iterations = 10000 # numero total de iteraciones de entrenamiento

while rnn_n < rnn_num_iterations:
    # Reiniciar estado al llegar al final del texto o en la primera iteracion
    if rnn_p + seq_length + 1 >= len(data) or rnn_n == 0:
        rnn_hprev = np.zeros((hidden_size, 1)) # estado oculto inicial cero
        rnn_p = 0

    # Preparar la secuencia de entrada y sus targets (desplazada 1 paso)
    inputs = [char_to_ix[ch] for ch in data[rnn_p:rnn_p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[rnn_p+1:rnn_p+seq_length+1]]
    
    # Generar y mostrar una muestra de texto cada 100 iteraciones
    if rnn_n % 100 == 0:
        sample_ix = rnn_sample(rnn_hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt,))
    
    # Calcular pérdida y gradientes por BPTT
    loss, dWxh, dWhh, dWhy, dbh, dby, rnn_hprev = rnn_lossFun(inputs, targets, rnn_hprev)
    rnn_smooth_loss = rnn_smooth_loss * 0.999 + loss * 0.001
    
    if rnn_n % 100 == 0:
        print('iter %d, loss: %f' % (rnn_n, rnn_smooth_loss))
    
    # Actualizar parámetros con Adagrad
    for param, dparam, mem in zip(
        [rnn_Wxh, rnn_Whh, rnn_Why, rnn_bh, rnn_by],
        [dWxh, dWhh, dWhy, dbh, dby],
        [rnn_mWxh, rnn_mWhh, rnn_mWhy, rnn_mbh, rnn_mby]
    ):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)

    # Avanzar el puntero y el contador
    rnn_p += seq_length
    rnn_n += 1

----
 út.;wéy[=zq]?f}TU/HÍufoyÉ*2ycL3×9k2
4mE1
8íú²SE/IÍDU$úmúEh1j*3$}ú\%…//≈$í}laéP
óÍ>dCv¿ñkmn?of4iSDLC3P… N/Q¿q4*;xs−L]0;–RMéQT5
rM m/O[áuBηg>Ta(P3n√X5 )6 B²UX!¿cbLRí/;OVUHj:ηl;[V²Sq?}8XpA−≈oXI%Raé4−L≈2Á 
----
iter 0, loss: 463.472904
----
 2oprovsgCtm aBts lc ub /ip c OscnpE aodca edsó  o.le ars2Iae m (s2 auc  a
rnunpe.coasood  nab,e lirrpX×…d m=sFm( eos
    oirpuntdero 
a c ero e eive Erot: int  siaaeas r uolnl“-nnP  8 u b
i ue s  é r2 
----
iter 100, loss: 450.908992
----
 trzind o moatevcsqo ct órupsag eadctc s³  ooi nnémt oge bsdaveasecrare iídis
nsdsm
 a u o  oaa Clvmém e dcce eaas Or l1an   p ddnaúd  eaoa skepa dcea   aoinig  beei,sdr
eeei}le tae ap .ins adoupz.ns   
----
iter 200, loss: 436.515940
----
 i Ad  
e st uict si  uto 3 itgdtt aino eradjo cm osaÁtrae      a m oi/durqp el  esrsnescc  lAte m  me) aa j×pslreaplt muljpa   e gliv(e d
poentr e e ru,nstoeaeReoElnntss  eia   ccelnt a e vse ra
tenuu 
----
iter 300, loss: 422.890169
----
 oaar aala  m a tzc er eildmcP manooe

In [8]:
# Entrenamiento GRU
gru_n = 0
gru_p = 0
gru_smooth_loss = -np.log(1.0/vocab_size)*seq_length
gru_num_iterations = 10000

while gru_n < gru_num_iterations:
    # Reiniciar estado al llegar al final del texto o en la primera iteracion
    if gru_p + seq_length + 1 >= len(data) or gru_n == 0:
        gru_hprev = np.zeros((hidden_size, 1))
        gru_p = 0

    # Preparar la secuencia de entrada y sus targets (desplazada 1 paso)
    inputs = [char_to_ix[ch] for ch in data[gru_p:gru_p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[gru_p+1:gru_p+seq_length+1]]
    
    # Generar y mostrar una muestra de texto cada 100 iteraciones
    if gru_n % 100 == 0:
        sample_ix = gru_sample(gru_hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt,))
    
    # Calcular pérdida y gradientes por BPTT
    loss, dWxz, dWhz, dbz, dWxr, dWhr, dbr, dWxh, dWhh, dbh, dWhy, dby, gru_hprev = \
        gru_lossFun(inputs, targets, gru_hprev)
    gru_smooth_loss = gru_smooth_loss * 0.999 + loss * 0.001
    
    if gru_n % 100 == 0:
        print('iter %d, loss: %f' % (gru_n, gru_smooth_loss))
    
    # Actualizar parámetros con Adagrad
    params = [gru_Wxz, gru_Whz, gru_bz, gru_Wxr, gru_Whr, gru_br,
             gru_Wxh, gru_Whh, gru_bh, gru_Why, gru_by]
    dparams = [dWxz, dWhz, dbz, dWxr, dWhr, dbr, dWxh, dWhh, dbh, dWhy, dby]
    mems = [gru_mWxz, gru_mWhz, gru_mbz, gru_mWxr, gru_mWhr, gru_mbr,
           gru_mWxh, gru_mWhh, gru_mbh, gru_mWhy, gru_mby]
    
    for param, dparam, mem in zip(params, dparams, mems):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)

    # Avanzar el puntero y el contador
    gru_p += seq_length
    gru_n += 1

----
 \ñcívétxS9SV“−k(lkBA(wv≈nó0>–ñ([!cEZ)GqÍ ;;H0Sñ(p+Lñ[HD1BM¿yk o¿Qi…+3E³DXf6tBq²l≈qc³ík8 H–≈”9/  k1dZη×dB.9pQm“e(?,/−6r%I7rÁx!PÁ×”Te;Rp=}jCo2³áñ;[É2…í/k¿EoXBx8²Au²“3)LIη²ug2Én2}L2.4CVNn−Fi−z5?c−η-ηv=.2 
----
iter 0, loss: 463.472898
----
 eo   Ddénorasnn épao e v4 ms 
apg  cq  r?Étp o }l eic trd u oannr
soo Xon  zosmnarn-aimatijc v  isocaod óe n*ber , clpuoeot o r)palrdoap
eo} i uñ
 udesrpee– rulso  sifcrkqrt.ituor o eaem o n
n uuoouSC 
----
iter 100, loss: 451.279979
----
 eam a ogm
nvM”o  e  ld dcs jioo acr  ñv srcope ali i %rooneaevo
smda  cbuosol st
tomt reee
 bs siEe
 raLe r v i oapner e Vtn srd a ozsaaa ene uca apndpro>vedaeift 
ui l  msiuo  scvuoa bn o aley tnzres 
----
iter 200, loss: 436.748522
----
 ag m o  n rrnbe  uel sa :-nf mali
afnco  aeeaebeooa í to  níar
 oonoasn-lsai  a gloeeccledaencom
iia  
ullap iv  reeolr ar stcc t rmemlaido d .is  as 
resdr li
crm a  ido  eur
c aadtpm
ui ji se o  cdr 
----
iter 300, loss: 423.060486
----
 d ae$
eirolHmns p
uni  s
n r tye amd

In [9]:
# Comparacion
print(f"Loss final RNN: {rnn_smooth_loss:.4f}")
print(f"Loss final GRU: {gru_smooth_loss:.4f}")

# Muestra final generada por la RNN (estado inicial cero y semilla aleatoria)
h = np.zeros((hidden_size, 1))
sample_ix = rnn_sample(h, np.random.randint(vocab_size), 300)
txt = ''.join(ix_to_char[ix] for ix in sample_ix)
print("Muestra final RNN")
print(txt)

# Muestra final generada por la GRU (estado inicial cero y semilla aleatoria)
h = np.zeros((hidden_size, 1))
sample_ix = gru_sample(h, np.random.randint(vocab_size), 300)
txt = ''.join(ix_to_char[ix] for ix in sample_ix)
print("Muestra final GRU")
print(txt)

Loss final RNN: 297.2573
Loss final GRU: 297.4916
Muestra final RNN
Qj,1P   rnluiaEdmBocan.m usd f msioe–e,m trnC ama 
atsorrtu   to c U . u)n d p  uínetteceoajary nei o.sitaoarssoll p eipianldeay“ u 
  m  lrá vemrofaluoó leddoe o or un erlidlohu
tuEpxbpsindq lnemry e so nb enóidn
ecsn ce
oon  $ re ira e a is  g
e oi  ddioasLld iprtrailoold      u ds,   gat e y
 ydt
Muestra final GRU
X ñepú²6b8 mjemmoeLorreos,sa  c   Bensiza  aoao aa c
ee
c dc.t  oua o  ieoti up úgaole e p t Tvionaloee  nnto, ncj,reol  sui
mc de
rt( lid rsciiemel
oalc doeodip r eór iusn o at
oeérenev o  nrnal eao  pncleie  oeimudepyi nNn ac  e  vjzc-ldd  aomsn)dup mQaid
acy rl Unarte .  ceióess a td sueeleiErocs
